# Module 4 Final Project

## Time Series Analysis of Real Estate  - Comparing New York City to San Francisco

The goal of this project is to provide insights and recommendations to our stakeholders, who are interested in learning what is the best return in real estate investment for a span of 2 and 5 years when deciding whether to buy on the East Coast in New York City, NY or in the West Coast in San Francisco, CA.

We will recommend, based on our analysis and modeling of historical data on real estate prices, obtained from the [Zillow Research Page](https://www.zillow.com/research/data/), what is the most sound financial decision, and we will also provide the 5 best zipcodes for buyers today in both cities for each investment time lag.


## Part I - Comparing San Francisco and NYC


### 1. Load libraries and data

We will start by loading all the necessary libraries, as well as our data from the file that has already been saved to this repository. 

In [ ]:
# importing relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly as py
import plotly.graph_objects as go
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from pyramid.arima import auto_arima
from pyramid.arima.utils import ndiffs
from sklearn.metrics import mean_squared_error
import math
import warnings
warnings.filterwarnings('ignore')
%matplotlib notebook

In [ ]:
# loading dataset
df = pd.read_csv('zillow_data.csv')
df.head()

In [ ]:
# checking how many unique values we have on each column, besides the time data
df.iloc[:,0:7].nunique()

### 2. Preparing the Data

Looking at our dataframe we can see that the time data is formatted as columns. We have both RegionID and RegionName, and a quick research online shows that the column RegionName is the one that refers to zipcodes. Let's rename it accordingly.

In [ ]:
# renaming column that refers to zipcodes
df.rename({'RegionName': 'Zipcode'}, axis='columns', inplace=True)
df.head()

We also have data for the entire country. Let's filter out only what we will need - New York City and San Francisco.

In [ ]:
# filtering only data for New York City and San Francisco
san_francisco = df.loc[df['City'] == 'San Francisco']
nyc = df.loc[df['City'] == 'New York']

In [ ]:
san_francisco.head()

We want to check how many unique zipcodes we have in each city.

In [ ]:
# checking total number of zip codes for SF
print(f'We have {san_francisco.Zipcode.nunique()} different zip codes in our dataset for San Francisco.')

In [ ]:
# checking total number of zip codes for NY
print(f'We have {nyc.Zipcode.nunique()} different zip codes in our dataset for NYC.')

Looks like we have a total of 133 unique zipcodes for our time series analysis. 

Our next step will be to use a function to transform our data from a wide format as it is now to a long format, and then make it into a pandas series of datetime object. We were provided with this helper function from our curriculum.

In [ ]:
# create function to melt data and make it into time series

def melt_data(df):
    ''' 
    Takes a dataframe with datetime data that is in wide format and melts it into long format; 
    Tranforms data into datetime object with time as index.
    User will need to change columns names on first line of code according to their own dataframe.
    '''
    
    melted = pd.melt(df, id_vars=['RegionID', 'Zipcode', 'City', 'State', 'Metro', 'CountyName', 'SizeRank'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

In [ ]:
# apply function to our data:
sf_ts = melt_data(san_francisco)
nyc_ts = melt_data(nyc)

Great, now our data is in a time series format with a DatetimeIndex and a column with our target values with a total of 252 datapoints, which correspond to 21 years. We can now move on to some visualizations of our data.

### 3. Visualizations

Now that we have our data in the right format for our two cities, let's start by looking at some visualizations to help us deepen our understanding and direct our analysis and modeling strategy.

Let's first of all do a quick visualization of this data, comparing both cities.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=sf_ts.index, y=sf_ts.value, name='San Francisco',
                         line_color='magenta'))
fig.add_trace(go.Scatter(x=nyc_ts.index, y=nyc_ts.value, name='New York City',
                         line_color='deepskyblue'))
fig.update_layout(title_text='San Francisco & NYC Zillow Home Value Index',
                  xaxis_rangeslider_visible=True)
fig.show()

That is very interesting. We can see from our graph that the mean Zillow Home Value Index (ZHVI) for the city of San Francisco, CA is higher than that for New York City, NY. We can also note the trends in the data for both cities: overall continuously rising trend, with some drop following the 2007-2008 real estate crisis - less so for NYC than for San Francisco. At the same time, we note that the upward trend is steeper for San Francisco during the last years - which makes sense when one thinks of the impact that the technology companies have been causing in that area.

We also note a big straight jump in the data for NYC in 2004, and upon investigating we see that this is because several zipcodes were created in that year. This inconsistency could make our modeling and predictions less robust but at the same time we could lose important information by eliminating zipcodes that could be rentable today, so we will choose to keep it that way for now.

From simply inspecting our data we can tell it is not stationary (positive trend), but we can go further into evaluating our data for trends and seasonality. We will create a function to calculate and plot rolling statistics for mean and standard deviation, as well as perform a Dickey-Fuller test for stationarity.

In [ ]:
# create function to plot time series with rolling mean and rolling variance, plus results from Dickey-Fuller test

def plot_roll(ts, name=''):
    
    '''Takes a time series and plots it along with its rolling mean and its rolling standard deviation.
    Calculates and prints results from Dickey-Fuller test.'''
    
    # Calculate rolling mean and rolling standard deviation:
    rolmean = ts.rolling(window = 6, center = False).mean()
    rolstd = ts.rolling(window = 6, center = False).std()
    
    # Plot original time series and its rolling mean/standard deviation
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=ts.index, y=ts.value, name='Original',
                             line_color='blue'))
    fig.add_trace(go.Scatter(x=rolmean.index, y=rolmean.value, name='Rolling Mean',
                             line_color='red'))
    fig.add_trace(go.Scatter(x=rolstd.index, y=rolstd.value, name='Rolling Std',
                             line_color='green'))
    fig.update_layout(title_text=f'{name} Rolling Mean & Standard Deviation',
                      xaxis_rangeslider_visible=True)
    fig.show();
    
    #Perform Dickey-Fuller test:
    print (f'{name} Results of Dickey-Fuller Test:')
    dftest = adfuller(ts['value'])

    # Extract and display test results in a user friendly manner
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    return print(dfoutput)

In [ ]:
# checking stationarity for San Francisco
sf_dfoutput = plot_roll(sf_ts, 'San Francisco')

In [ ]:
# checking stationarity for NYC
plot_roll(nyc_ts, 'New York')

From our rolling statistics plots we can note that even though we can't easily observe a trend or seasonality in our standard deviation, the mean is notably increasing so our data is clearly not stationary. Of course the results from the Dickey-Fuller tests also confirm this, for both cities. We will, further ahead, need to make it stationary in order to model it correctly.


We want to create a yearly dataframe so that we check for any seasonality throughout the years. For this we'll create a function that takes a time series and returns a plot with the yearly grouped data.

In [ ]:
#Create a new DataFrame with yearly values stored in columns 

def plot_yearly_ts(ts, name='', boxplot=False):
    
    '''Function takes a time series and groups it into yearly intervals using Grouper;
    Creates a new dataframe where yearly values are into columns to faciliate plotting.
    Gives the option to create a boxplot instead of line plot, if parameter is passed as True.'''
    
    # group data by year and made into dataframe
    year_group = ts.groupby(pd.Grouper(freq ='A'))
    yearly = pd.DataFrame()
    for year, group in year_group:
        yearly[year.year] = group.values.ravel()
        
    # plot boxplot if option is True
    if boxplot:
        ax = yearly.boxplot(figsize = (18,10))
        ax.set_ylabel('Value($)')
        ax.set_xlabel('Year')
        plt.title(f'{name} Mean Zillow Home Value Index (ZHVI) - Yearly', fontsize=18);
        
   # otherwise plot line plot
    else:
        plt.figure(figsize = (14,10))
        ax = plt.subplot(111)
        # speficy colors for plots
        tableau20 = [(0.12156862745098039, 0.4666666666666667, 0.7058823529411765), (0.6823529411764706, 0.7803921568627451, 0.9098039215686274), (1.0, 0.4980392156862745, 0.054901960784313725), (1.0, 0.7333333333333333, 0.47058823529411764), (0.17254901960784313, 0.6274509803921569, 0.17254901960784313), (0.596078431372549, 0.8745098039215686, 0.5411764705882353), (0.8392156862745098, 0.15294117647058825, 0.1568627450980392), (1.0, 0.596078431372549, 0.5882352941176471), (0.5803921568627451, 0.403921568627451, 0.7411764705882353), (0.7725490196078432, 0.6901960784313725, 0.8352941176470589), (0.5490196078431373, 0.33725490196078434, 0.29411764705882354), (0.7686274509803922, 0.611764705882353, 0.5803921568627451), (0.8901960784313725, 0.4666666666666667, 0.7607843137254902), (0.9686274509803922, 0.7137254901960784, 0.8235294117647058), (0.4980392156862745, 0.4980392156862745, 0.4980392156862745), (0.7803921568627451, 0.7803921568627451, 0.7803921568627451), (0.7372549019607844, 0.7411764705882353, 0.13333333333333333), (0.8588235294117647, 0.8588235294117647, 0.5529411764705883), (0.09019607843137255, 0.7450980392156863, 0.8117647058823529), (0.6196078431372549, 0.8549019607843137, 0.8980392156862745), (0.6823529411764706, 0.7803921568627451, 0.9098039215686274), (1.0, 0.4980392156862745, 0.054901960784313725), (1.0, 0.7333333333333333, 0.47058823529411764)]
        # remove borders from plot
        ax.spines["top"].set_visible(False)  
        ax.spines["right"].set_visible(False)
        n_points = yearly.shape[0]
        for rank, year in enumerate(yearly):
            ax.plot(np.arange(n_points-(n_points-1), n_points+1), yearly[year].values, color=tableau20[rank], lw=2)
            plt.xlim(1, 12)
            y_pos = yearly[year].values[-1] 
            ax.text(n_points, y_pos, year, color=tableau20[rank], fontsize=12)
            ax.grid(which='major', axis='y', linestyle= "--", lw=0.5, color="black", alpha=0.3)
            ax.set_ylabel('Value($)')
            ax.set_xlabel('Month')
            plt.title(f'{name} Mean Zillow Home Value Index (ZHVI) - Yearly', fontsize=18)

In [ ]:
# check data
sf_ts.head(), sf_ts.tail()

While trying to run our function we were getting errors and upon investigating our data further, we noticed that both years 1996 and 2018 are not complete years, so we will filter out these years and keep only the years that have full monthly data for the yearly plots. 

In [ ]:
# filter out years that are not complete
sf_ts_filter = sf_ts['1997':'2017']
nyc_ts_filter = nyc_ts['1997':'2017']

In [ ]:
# plot yearly data for San Francisco
plot_yearly_ts(sf_ts_filter, 'San Francisco')

That is a lot of data in one plot, but it is useful to note a few things. We can't observe any clear seasonality in the yearly data, and we can see that the mean steadily moves up as the years increase, as we were able to observe in the previous plot, except for the years between 2009 and 2011 which reflect the impact of the financial crisis. Let's do a second visualization of only those years during the housing bubble crisis.

In [ ]:
# plot other years for San Francisco for the time of the financial crisis
plot_yearly_ts(sf_ts_filter['2007':'2012'], 'San Francisco')

Here in this plot we can see that, starting at the end of 2007 home values started to fall and continued to do so  for the next 4 years, until about the end of 2011 when the trend starts to move up again. It was a slow fall and a quick recovery - values were at the lowest around August 2011 and by the end of 2012 house values are almost at the same level they were at the start of the crisis.

Let's also look at the boxplots for this data.

In [ ]:
# plot boxplots for San Francisco
plot_yearly_ts(sf_ts_filter, 'San Francisco', boxplot=True)

The boxplot further confirms that there is a positive trend in the data. It also shows that the data variance is not changing by much year by year, and there is less variance coinciding with the years with more of a flat trend. We can again see that the mean is rising, except for the years between 2008 and 2011. There are also not many outliers in our data.  

Let's also have a look at New York City.

In [ ]:
# plot yearly data for NYC 
plot_yearly_ts(nyc_ts_filter, 'New York City')

Again we see that there doesn't seem to be any clear seasonality in our data. As with San Francisco, the values steadily increase except for the period between 2006 and 2012 when values are much closer together. Let's again have a zoomed up look at these years only.

In [ ]:
# plot other years data for NYC
plot_yearly_ts(nyc_ts_filter['2006':'2012'], 'New York City')

What we can obserse is that for NYC the effects of the housing crisis were less steady. We see how by middle of 2006 prices were rising up (as it was with the bubble for most locations) and then start to fall quickly, but by end of 2007 values were rising again, only to fall in 2008 - but never as low as they were in the end of 2006. By 2013 the recovery is clear and seems exponential. Overall this tells us that house values in NYC fared better during the crisis overall than that of San Francisco.

Let's also look at the boxplot for the New York City data.

In [ ]:
# plot boxplots for NYC
plot_yearly_ts(nyc_ts_filter, 'New York City', boxplot=True)

The New York City data looks similar, and we can note that the financial crisis had less impact for NYC. The values steadily rise from year to year with some stagnation in the wake of the house bubble crisis, from years 2009 to 2012. 

### 4. Calculate Returns Statistics

We want to create some statistics to help us better evaluate returns and eventual losses in times of crises for both cities, which is what our stakeholders are interested in. 

We'll create a function in order to do that. Our function will calculate what was the percentage gain over amount invested in a lag of 2, 5 and 10 years previous, throughout the time, plot and return the information as dataframes for further investigation and modeling.

In [ ]:
# function to calculate gain percentage (ROI) on previous 2, 5 and 10 years
def calculate_gain(ts, plot=True):
    
    '''Takes a real estate time series and performs calculations on returns over investments
    for periods of 2, 5 and 10 years.'''
    
    # calculates ROI by taking current value, decreasing investment(value at x steps in past) and 
    # dividing by investment. Multiplies by 100 to get percentage number
    roi_2 = (ts - ts.shift(periods=24))/ts.shift(periods=24)*100
    roi_5 = (ts - ts.shift(periods=60))/ts.shift(periods=60)*100
    roi_10 = (ts - ts.shift(periods=120))/ts.shift(periods=120)*100
    roi_2.dropna(inplace=True)
    roi_5.dropna(inplace=True)
    roi_10.dropna(inplace=True)
    
    # plot results
    if plot:
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=roi_2.index, y=roi_2.value, name='2-Year Investment',
                                 line_color='deepskyblue'))
        fig.add_trace(go.Scatter(x=roi_5.index, y=roi_5.value, name='5-Year Investment',
                                 line_color='red'))
        fig.add_trace(go.Scatter(x=roi_10.index, y=roi_10.value, name='10-Year Investment',
                                 line_color='gray'))
        fig.update_layout(title_text='Mean Zillow Home Value Index (ZHVI) - ROI Over Time Invested',
                          xaxis_rangeslider_visible=True)
        fig.show()
    
    # saves new ts with returns for 2, 5 and 10 periods
    return roi_2, roi_5, roi_10

In [ ]:
# let's call the function to examine San Francisco's data first
sf_roi2, sf_roi5, sf_roi10 = calculate_gain(sf_ts)

Not surprisingly, the returns for investment with a time lag of 10 years are potentially the highest - they peaked at just over 200% at around 2007 in San Francisco for the people who had bought 10 years before. It is also at the highest fluctuation, as it comes down quickly form over 200% to around 20% in a span of approximately 5 years. It is however always a positive return, never under 20% even at the height of financial crisis.

On the other hand, on a 2 and 5 lag investment, there is loss during the financial crisis, from around 2008 to 2012-2013, that reaches a little over 12% of loss. Once over the crisis the gains rise again, but are now in what looks a downward trend. 

Let's see how NYC fares.

In [ ]:
nyc_roi2, nyc_roi5, nyc_roi10 = calculate_gain(nyc_ts)

In NYC one thing we notice is the effect of that quick drop that shows the period in time (around 2004) with the creation of new zipcodes. Regardless of this, what we can see is that investing in NYC has rarely represented a loss, even at the peak of the housing crisis as we have only a short period of loss around 2010 and 2013. The returns on 10 year-investments also seem highest for NYC, reaching over 260% at the initial point, and even at the lowest return in 10-years it is around 40% and thus higher when compared to the lowest 10-year return for San Francisco (of around 20%).

New York also seems to be starting to have a downward trend in terms of return of investment, although it doesn't seem as pronounced as for San Francisco.

We want to further compare the returns for both cities, side-by-side. We'll create a quick function to return the max and min return for each city for each investment lag.

In addition we want to plot the cities side-by-side so that we can better visualize and compare their performances.

In [ ]:
# create visualization of both time series:
plt.figure(figsize=(18,6))
ax = plt.subplot(111)  
ax.spines["top"].set_visible(False)  
ax.spines["right"].set_visible(False)
ax.plot(sf_roi10, label='San Francisco', color='magenta')
ax.plot(nyc_roi10, label='NYC', color='blue')
ax.fill_between(sf_roi10.index, sf_roi10.value<0, -20, color='red', alpha=.2)
plt.title('San Francisco vs. NYC - Return on 10 year Investment', fontsize=16)
ax.set_ylabel('Gain Percentage')
ax.set_xlabel('Year')
plt.yticks([-10,0,10,25,50,75,100,125,150,175,200, 250], [str(x) + "%" for x in [-10,0,10,25,50,75,100,125,150,175,200, 250]], fontsize=10)
ax.grid(which='major', axis='y', linestyle= "--", lw=0.5, color="black", alpha=0.3)
ax.legend()
plt.show();

It looks like over time investing in NYC has brought higher returns, and this only reverses more recently, from around 2015, which is a clear reflection of the current real estate situation in San Francisco, where there is a big housing shortage following the tech and Internet job boom in that area. From [Wikipedia](https://en.wikipedia.org/wiki/San_Francisco_housing_shortage): "from 2012 to 2016, the San Francisco metropolitan area added 373,000 new jobs, but permitted only 58,000 new housing units."

Even so, the return in investment for both cities is very close at current times. However, we must point out that the mean house price for New York City is less than for San Francisco, meaning that less money invested is needed. Let's see how they compare for 2-year and 5-year investments over time.

In [ ]:
# create visualization of both time series:
plt.figure(figsize=(18,6))
ax = plt.subplot(111)  
ax.spines["top"].set_visible(False)  
ax.spines["right"].set_visible(False)
ax.plot(sf_roi5, label='San Francisco', color='magenta')
ax.plot(nyc_roi5, label='NYC', color='blue')
ax.fill_between(sf_roi5.index, sf_roi5.value<0, -20, color='red', alpha=.2)
plt.title('San Francisco vs. NYC - Return on 5 year Investment', fontsize=16)
ax.set_ylabel('Gain Percentage')
ax.set_xlabel('Year')
plt.yticks([-10,0,10,25,50,75,100,125,150,175,200, 250], [str(x) + "%" for x in [-10,0,10,25,50,75,100,125,150,175,200, 250]], fontsize=10)
ax.grid(which='major', axis='y', linestyle= "--", lw=0.5, color="black", alpha=0.3)
ax.legend()
plt.show();

In [ ]:
# create visualization of both time series:
plt.figure(figsize=(18,6))
ax = plt.subplot(111)  
ax.spines["top"].set_visible(False)  
ax.spines["right"].set_visible(False)
ax.plot(sf_roi2, label='San Francisco', color='magenta')
ax.plot(nyc_roi2, label='NYC', color='blue')
ax.fill_between(sf_roi2.index, sf_roi2.value<0, -20, color='red', alpha=.2)
plt.title('San Francisco vs. NYC - Return on 2 year Investment', fontsize=16)
ax.set_ylabel('Gain Percentage')
ax.set_xlabel('Year')
plt.yticks([-10,0,10,25,50,75,100,125,150,175,200, 250], [str(x) + "%" for x in [-10,0,10,25,50,75,100,125,150,175,200, 250]], fontsize=10)
ax.grid(which='major', axis='y', linestyle= "--", lw=0.5, color="black", alpha=0.3)
ax.legend()
plt.show();

What we can see from both plots is that San Francisco started out in our time frame as a better investment, was overtaken by NYC right before the crisis, and are now more comparable, especially on a 2-year lag. 

Let's do a little side-by-side comparison for highest and lowest returns for each city so that we can more easily see which is the city that shows the best returns and smaller losses. I'll make another function to get this statistics comparison in an easy to read format.

In [ ]:
def compare_gain(ts1, ts2, name='', name2='', plot=True):
    
    '''Takes two time series, calculates returns in 2, 5 and 10 years for each 
    and returns max and min values for each'''
    
    # calculates returns for first time series
    ts1.dropna(inplace=True)
    ts1_roi_2, ts1_roi_5, ts1_roi_10 = calculate_gain(ts1, plot=False)

    
    # calculates returns for second time series
    ts2.dropna(inplace=True)
    ts2_roi_2, ts2_roi_5, ts2_roi_10 = calculate_gain(ts2, plot=False)
    
    # organize returns values and make into dataframes
    hist_roi = {}
    hist_roi[name] = {}
    hist_roi[name]['Max Return 02-Year'] = round(ts1_roi_2.values.max(),2)
    hist_roi[name]['Min Return 02-Year'] = round(ts1_roi_2.values.min(),2)
    hist_roi[name]['Max Return 05-Year'] = round(ts1_roi_5.values.max(),2)
    hist_roi[name]['Min Return 05-Year'] = round(ts1_roi_5.values.min(),2)
    hist_roi[name]['Max Return 10-Year'] = round(ts1_roi_10.values.max(),2)
    hist_roi[name]['Min Return 10-Year'] = round(ts1_roi_10.values.min(),2)
    
    hist_roi[name2] = {}
    hist_roi[name2]['Max Return 02-Year'] = round(ts2_roi_2.values.max(),2)
    hist_roi[name2]['Min Return 02-Year'] = round(ts2_roi_2.values.min(),2)
    hist_roi[name2]['Max Return 05-Year'] = round(ts2_roi_5.values.max(),2)
    hist_roi[name2]['Min Return 05-Year'] = round(ts2_roi_5.values.min(),2)
    hist_roi[name2]['Max Return 10-Year'] = round(ts2_roi_10.values.max(),2)
    hist_roi[name2]['Min Return 10-Year'] = round(ts2_roi_10.values.min(),2)
    
    last_roi = {}
    last_roi[name] = {}
    last_roi[name]['Mean 6M Investment'] = round(ts1.iloc[-7:-1].values.mean(),0)
    last_roi[name]['Mean 6M Return 02-Year'] = round(ts1_roi_2.iloc[-7:-1].values.mean(),2)
    last_roi[name]['Mean 6M Return 05-Year'] = round(ts1_roi_5.iloc[-7:-1].values.mean(),2)
    last_roi[name]['Mean 6M Return 10-Year'] = round(ts1_roi_10.iloc[-7:-1].values.mean(),2)
    
    last_roi[name2] = {}
    last_roi[name2]['Mean 6M Investment'] = round(ts2.iloc[-7:-1].values.mean(),0)
    last_roi[name2]['Mean 6M Return 02-Year'] = round(ts2_roi_2.iloc[-7:-1].values.mean(),2)
    last_roi[name2]['Mean 6M Return 05-Year'] = round(ts2_roi_5.iloc[-7:-1].values.mean(),2)
    last_roi[name2]['Mean 6M Return 10-Year'] = round(ts2_roi_10.iloc[-7:-1].values.mean(),2)
           
    hist = pd.DataFrame.from_dict(hist_roi)
    last = pd.DataFrame.from_dict(last_roi)
    complete = pd.concat([hist, last])
    
    
    # plot results
    if plot:
        my_colors = [(x/10.0, x/20.0, 0.75) for x in range(1,12)]
        ax = complete.loc[['Mean 6M Investment'], :].plot(kind='bar', colormap='Accent', figsize=(8,6))
        ax.set_xlabel('Mean 6M Investment')
        ax.set_xticks([])
        plt.title('Mean Last 6 Months Investment', fontsize=16)
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        ax.spines["top"].set_visible(False)  
        ax.spines["right"].set_visible(False)
        
        gains = complete.drop(['Mean 6M Investment','Mean 6M Return 02-Year','Mean 6M Return 05-Year','Mean 6M Return 10-Year'], axis=0)
        ax = gains.plot(kind='bar', colormap='Accent', figsize=(12,8))
        plt.gcf().autofmt_xdate()
        plt.title('Historical Returns', fontsize=16)
        plt.axhline(y=0, color='red')
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        ax.spines["top"].set_visible(False)  
        ax.spines["right"].set_visible(False)
        
        ax = complete.loc[['Mean 6M Return 02-Year','Mean 6M Return 05-Year','Mean 6M Return 10-Year'], :].plot(kind='bar', colormap='Accent', figsize=(8,6))
        plt.gcf().autofmt_xdate()
        plt.title('Mean Last 6 Months Return', fontsize=16)
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        ax.spines["top"].set_visible(False)  
        ax.spines["right"].set_visible(False)

    
    return complete 
    

In [ ]:
compare_gain(sf_ts, nyc_ts, name='San Francisco', name2='New York City')

From this comparison data, New York is historically the best option to invest, regardless of the time-lag for the investment as it has provided the highest return rate and smaller loss when that is the case for all time options: 2, 5 and 10-year investments. It also requires less capital invested. 

However, the return in the last period of our data, the month of 2007 were higher for San Francisco. In any case, we want to model the returns data so that we can see what the prediction is for the future in terms of not only house price but rather of returns over investment, and if the trend is favorable to investing in San Francisco, despite the history showed by the data.



### 5. Modeling

Our stakeholders are interested in investing with a 2 and 5 year return, so we will only work with these two series for our modeling and prediction. In any case, predictions for over 5 years would be too far ahead and not reliable (have a too wide confidence interval).

As a first step in our modeling process, we will create a function to calculate and plot both the Autocorrelation (ACF) and the Partial Autocorrelation (PACF) for our data, which will be helpful with some initial idea for difference order we might use, as well as the other parameters of our model. Let's also check our returns data for stationarity.

In [ ]:
# checking stationarity 
returns = [sf_roi2, sf_roi5, nyc_roi2, nyc_roi5]
names = ['San Francisco 2-Year', 'San Francisco 5-Year', 'NYC 2-Year', 'NYC 5-Year']
for ts, name in zip(returns, names):
    plot_roll(ts, name=name)

Most of our data is not stationary and this means we will need to difference it by at least order 1 in order to better fit a model. However, we note that the spike with the creation of new zipcodes in NYC in 2004 is causing our data to look very inconsistent and this might be prejudicial to our modeling. We want to remove this data impacted so as to not confuse our models, and work with only the years after such impact - 2006 for the 2-Year data, and 2009 for the 5-Year data. Let's remove this and plot the rolling statistics again for NYC.

In [ ]:
# filtering out inconsistent data for New York City
nyc_roi2 = nyc_roi2['2006':]
nyc_roi5 = nyc_roi5['2009':]

# save variable again to make sure we have the current time series for NYC
returns = [sf_roi2, sf_roi5, nyc_roi2, nyc_roi5]

In [ ]:
plot_roll(nyc_roi2, 'New York 2-Year')

In [ ]:
plot_roll(nyc_roi5, 'New York 5-Year')

 That is much better. Let's look at all ACF and PACF plots. We will create a function to take a time series and plot both ACF and PACF at once.

In [ ]:
# create function to plot ACF and PACF
def acf_pacf(ts,alags=40,plags=40):
    fig,(ax1,ax2) = plt.subplots(2,1,figsize=(14,8))
    plot_acf(ts,lags=alags, zero=True,ax=ax1)
    plot_pacf(ts,lags=plags, ax=ax2, method = 'ywmle')
    plt.show();

In [ ]:
# checking ACF and PACF plots  
for df, name in zip(returns, names):
    print(f'\n \n {name} \n')
    acf_pacf(df)

We can see on our ACF plots that for most data there is a significant positive correlation up to around 15 lags, which is expected for this type of data such as real estate values. This might give us a hint that this number could be a good order for our MA parameter. 

On our PACF plots we see that we might be dealing with an AR order 4 or 5 for the San Francisco 2-Year Return data, and likely 1 or 2 for the others. 

We also want to decompose the data so that we can note if there is any seasonality that we can't observe clearly.

In [ ]:
# decompose time series
for df in returns:
    result = seasonal_decompose(df, model='additive')
    result.plot()

It looks like indeed there is some seasonality for all the data, so we will keep this in mind. We will work with a SARIMAX model to account for all factors. As a first step we will work with the data for San Francisco 2-year Returns for a test, and run an initial model as our base, using order 1 for all parameters and s=12.

In [ ]:
def fit_sarimax_model(ts, order=(1,1,1), seasonal_order=(0, 0, 0, 12), summary=True, plot=True):
    
    '''Takes a time series and runs a SARIMAX model with parameter order provided. 
    If no parameter provided default is (1,1,1), (0,0,0,0). 
    Has the default option to print model summary and plot diagnostics, which can be turned off'''
  
    # fit model
    model = sm.tsa.statespace.SARIMAX(ts,
                                    order=order,
                                    seasonal_order=seasonal_order,
                                    trend='ct',
                                    enforce_stationarity=False,
                                    enforce_invertibility=False)
    output = model.fit(d=0)
    
    if summary==True:
        print(output.summary())

    if plot==True:
        # plot model diagnostics
        output.plot_diagnostics(figsize=(15, 18))
        plt.show()
        
    return output

In [ ]:
sf2_output = fit_sarimax_model(sf_roi2)

We can see from our diagnostics that our residuals are not quite normally distributed, and show some correlation so that means our model is far from ideal and we must work to improve it. That makes sense since this was a basic, initial model.

Let's get predictions and the RMSE for this model so that we have some comparison ahead. We will again create a function to make this step easily replicated.

In [ ]:
def get_predictions(ts, model_output, steps=24, plot=True, show=True):
    
    '''Gets one-step-ahead forecast for model, 
    calculates Root of Mean Squared Error, 
    makes future predictions for number
    of steps passed as parameter(default is 24), 
    plots results, 
    provides last forecasted value with confidence interval'''

    
    # get preditions from model for data period
    pred = model_output.get_prediction(start='2016-01-01', dynamic=True, full_results=True)
    conf = pred.conf_int()
    
    if plot:
        #Plot observed and predicted values with confidence interval
        ax = ts['2014':].plot(label='Observed')
        pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.9)
        ax.fill_between(conf.index,
                        conf.iloc[:, 0],
                        conf.iloc[:, 1], color='g', alpha=.2,
                        label='Confidence Interval')
        ax.set_ylabel('Return %')
        plt.title('Observations vs Predictions')
        ax.legend()
        plt.show()

    # compare real and predicted values to validade model and compute the rmse
    predicted = pred.predicted_mean
    real = ts['2016-01-01':].value
    mse = mean_squared_error(real, predicted)
    rmse = math.sqrt(mse)
    
    if show:
        print(f'The RMSE of our forecast is {round(rmse, 2)}.' + '\n')
        
    
    # Get forecast and confidence interval for steps ahead in future
    future = model_output.get_forecast(steps=steps, dynamic=True)
    future_conf = future.conf_int(steps=steps)

    if plot:
        # plot results
        ax = ts['2014':].plot(label='Observed', figsize=(12, 8))
        future.predicted_mean.plot(ax=ax, label='Forecast')
        ax.fill_between(future_conf.index,
                        future_conf.iloc[:, 0],
                        future_conf.iloc[:, 1], color='k', alpha=.25)
        ax.set_xlabel('Date')
        ax.set_ylabel('Returns')
        ax.legend()
        plt.show()

    # show prediction for end of step-period (in this case in 2 years future time)
    forecast = future.predicted_mean[-1]
    maximum = future_conf.iloc[-1,1]
    minimum = future_conf.iloc[-1,0]
    predictions = {}
    predictions['forecast'] = forecast
    predictions['maximum'] = maximum
    predictions['minimum'] = minimum
    
    predictions = pd.DataFrame.from_dict(predictions, orient='index', columns=['Return at End of Forecast'])
    
    if show:
        print(predictions)
        
    return forecast, maximum, minimum

In [ ]:
sf2_predictions = get_predictions(sf_roi2, sf2_output)

Our initial model predicts that within 2 years from the end of our data, the return for investment for those who had invested 2 years prior would be of about 29%, but the wide confidence intervals put this return ranging from -7.6% to 66%.

This is a large interval for our prediction and wouldn't help much our stakeholders with their decision. Let's try to fit a model using the parameters that we could infer from our ACF and PACF plots and see how it performs.

In [ ]:
sf2_output2 = fit_sarimax_model(sf_roi2, order=(3, 1, 14), seasonal_order=(0, 0, 0, 12))

In [ ]:
get_predictions(sf_roi2, sf2_output2)

Our AIC is much smaller, but the RMSE is higher. This means we might possibly be overfitting. In addition, our model is very complex with so many lags. 

In order to find the better order parameters for our model, we will use the function auto_arima, which performs a search over possible orders and helps us select the parameters that minimize a given metric. In this case we are using the AIC (Akaike Information Criterion) value to measure the quality of our model.

We will input the maximum parameters that we could infer from our ACF and PACF plots, and leave the function to estimate automatically the differencing term.

(You can learn more about the auto_arima [here](http://www.alkaline-ml.com/pmdarima/0.9.0/modules/generated/pyramid.arima.auto_arima.html).)

In [ ]:
# define a function to run auto arima and search for best model parameters
def find_orders(ts, name='', show=True):
    
    '''Takes a time series and finds the best differencing order, as well as the other best
    parameters for a SARIMAX model using auto_arima'''
    
    stepwise_model = auto_arima(sf_roi2, start_p=1, start_q=1,
                               max_p=5, max_q=15, seasonal=True,
                               m=12, test='adf', trace=False,
                               error_action='ignore',  
                               suppress_warnings=True, 
                               stepwise=True)
    if show:
        print(f'{name} - Order: {stepwise_model.order}, Seasonal Order" {stepwise_model.seasonal_order}, AIC: {stepwise_model.aic()}')
        
    return stepwise_model.order, stepwise_model.seasonal_order

In [ ]:
find_orders(sf_roi2, name='', show=True)

Great! Now let's fit our model again using these parameters provided and see how we perform.

In [ ]:
sf2_output3 = fit_sarimax_model(sf_roi2, order=(4, 0, 4), seasonal_order=(2, 0, 0, 12))

This model does seem better. We have a much lower AIC than our base, and the residuals look a bit more normally distributed. Let's verify what predictions and errors we get with this new model.

In [ ]:
get_predictions(sf_roi2, sf2_output3)

This model indeed seems like it does a better job than our previous tries. The RMSE for our forecast is smaller and our prediction range is not as wide, with a 12% return predicted but possibly ranging from -10% to 34% at a 95% confidence interval.

What we want to do now is to streamline this process and apply it to all our time series so that we can compare which one is the best. Let's create functions to do all this at once.

In [ ]:
# create function to compare 2-year returns
def compare_2y_roi(ts_list, names_list, plot=True):
    
    '''
    Takes a list of time series and its names and returns forecasted returns, along with
    confidence interval values for a 2-year in time prediction in a dataframe format.
    '''
    # create dictionary to store results
    hist_roi = {}   
    
    # model and get predictions
    for ts, name in zip(ts_list, names_list):
        hist_roi[name] = {}
        order, seasonal_order = find_orders(ts, show=False)
        model = fit_sarimax_model(ts, order=order, seasonal_order=seasonal_order, summary=False, plot=False)
        forecast, maximum, minimum = get_predictions(ts, model, steps=24, plot=False, show=False)
        hist_roi[name]['Predicted Return'] = forecast
        hist_roi[name]['Maximum Return'] = maximum
        hist_roi[name]['Minimum Return'] = minimum
    
    # format data into dataframe
    results = pd.DataFrame.from_dict(hist_roi)
    
    # plot results
    if plot:
        ax = results.plot(kind='bar', colormap='Accent', figsize=(8,6))
        plt.axhline(y=0, color='red')
        plt.title('Projected Returns after 2-Year Period', fontsize=16)
        ax.spines["top"].set_visible(False)  
        ax.spines["right"].set_visible(False)
        plt.gcf().autofmt_xdate()
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        
    return results
        

In [ ]:
# define another function for a 5-Year time return
def compare_5y_roi(ts_list, names_list, plot=True):
        
    '''
    Takes a list of time series and its names and returns forecasted returns, along with
    confidence interval values for a 5-year in time prediction in a dataframe format.
    '''
    # create dictionary to store results
    hist_roi = {}
    
    # model and get predictions
    for ts, name in zip(ts_list, names_list):
        hist_roi[name] = {}
        order, seasonal_order = find_orders(ts, show=False)
        model = fit_sarimax_model(ts, order=order, seasonal_order=seasonal_order, summary=False, plot=False)
        forecast, maximum, minimum = get_predictions(ts, model, steps=60, plot=False, show=False)
        hist_roi[name]['Predicted Return'] = forecast
        hist_roi[name]['Maximum Return'] = maximum
        hist_roi[name]['Minimum Return'] = minimum
        
    # format data into dataframe
    results = pd.DataFrame.from_dict(hist_roi)
    
    # plot results
    if plot:
        ax = results.plot(kind='bar', colormap='Accent', figsize=(8,6))
        plt.axhline(y=0, color='red')
        plt.title('Projected Returns after 5-Year Period', fontsize=16)
        ax.spines["top"].set_visible(False)  
        ax.spines["right"].set_visible(False)
        plt.gcf().autofmt_xdate()
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        
    return results

In [ ]:
ts_2year = [sf_roi2, nyc_roi2]
names_2year = ['San Francisco', 'New York City']
forecast_2years = compare_2y_roi(ts_2year, names_2year)
forecast_2years

In [ ]:
ts_5year = [sf_roi5, nyc_roi5]
names_5year = ['San Francisco', 'New York City']
compare_5y_roi(ts_5year, names_5year)

This is a very interesting result! The past data had shown us that New York City has been consistently a better city to invest in real estate when compared to San Francisco, regardless of the horizon of the return: 2, 5 or 10 years. However, the most recent data from our time series shows San Francisco returns above those of from NYC. We wanted to see if this was a trend that would be sustained in the future for our stakeholders, making San Francisco a more interesting investment option in despite of New York historical superior gains.

What our model results show us is that if our stakeholders are looking to buy and sell within 2 years, San Francisco indeed sounds like the best option for investors that are not risk averse, with higher predicted and maximum outcomes. However, San Francisco's predicted minimum return is more than that of New York City. We can conclude that San Francisco offers more gain potential however presents higher risk. 

On the other hand, when the timespan for the investment extends to 5 years, the predictions for San Francisco are much more volative. There could be a loss as high as 70% and the predicted return is much smaller than that of New York City. The later shows a more consistent investment option, with high predicted gains and low risk when the time of investment is of 5 years.

In conclusion:

### **For a 2-Year Investment:** 

Agressive/Focus on higher potential gain: **San Francisco** <br>
Conservative/Focus on smaller potential loss: **New York City**

### **For a 5-Year Investment:** 

**New York City**

Ok, now that we have some recommendations for our stakeholders in terms of which city to invest, let's move on to the second part of the project and find the best 5 zipcodes in each location to invest on.

## Part II - Best Zipcodes to Invest

Besides the overal analysis comparing these two cities, we also want to recommend what are the best zipcodes to buy real estate in each place. For that we will create a dictionary with the time series data for all zipcodes from both cities so that we have access to the data for all zipcodes in order to model.

In [ ]:
# Dictionary to include all zip code time series
sf_zip_dict = {}
    
# Iterate over all the zip codes within each city
for z in san_francisco.Zipcode.unique():
    
    # Choosing that specific zip code
    temp_zip_df = san_francisco[san_francisco.Zipcode == z]

    # Creating a time series (via melting) for that zip code
    temp_zip_ts = melt_data(temp_zip_df)

    # Adding that time series to a dictionary
    sf_zip_dict[z] = temp_zip_ts


In [ ]:
# we can now access each zip code by the keys
sf_zip_dict.keys()

In [ ]:
sf_zip_dict[94109].head()

In [ ]:
# we can also plot them
fig = go.Figure()
for i, key in enumerate(sf_zip_dict):
    fig.add_trace(go.Scatter(x=sf_zip_dict[key].index, y=sf_zip_dict[key].value, name=str(list(sf_zip_dict.keys())[i-1])))
fig.update_layout(title_text='San Francisco Zillow Home Value Index (ZHVI)',
                      xaxis_rangeslider_visible=False, showlegend=False)
fig.show()

We can see that some zip codes did suffer more harshly from the crisis, which can indicate higher risk for investments. We can also note that the rising trend is more accute for some zip codes when compared to others, which could lead to higher ROIs for the short/medium timeframe. 

Let's do the same thing for the New York City zip codes, generating the time series for each one and saving it all into a dictionary.

In [ ]:
# repeat with nyc data:
nyc_zip_dict = {}
    
# Iterate over all the zip codes within each city
for z in nyc.Zipcode.unique():
    
    # Choosing that specific zip code
    temp_zip_df = nyc[nyc.Zipcode == z]

    # Creating a time series (via melting) for that zip code
    temp_zip_ts = melt_data(temp_zip_df)

    # Adding that time series to a dictionary
    nyc_zip_dict[z] = temp_zip_ts

In [ ]:
# plot zipcodes data all together
fig = go.Figure()
for i, key in enumerate(nyc_zip_dict):
    fig.add_trace(go.Scatter(x=nyc_zip_dict[key].index, y=nyc_zip_dict[key].value, name=str(list(nyc_zip_dict.keys())[i-1])))
fig.update_layout(title_text='New York Zillow Home Value Index (ZHVI)',
                      xaxis_rangeslider_visible=False, showlegend=False)
fig.show()

Interesting plot that we should never show to a stakeholder. In any case it reminds us that some NYC zipcodes were created later on. Let's remove all NaNs from our data so that we don't encounter any problems with our modeling further ahead.

In [ ]:
# drop NaNs
for key in nyc_zip_dict:
    nyc_zip_dict[key].dropna(inplace=True, axis=0)

In [ ]:
# check data
nyc_zip_dict[11223].isna().sum()

All right, now we need a function to do the heavy lifting for us. We want to calculate the 2-Year and the 5-Year return for each of these zipcodes, and then select the 5 best. The criteria that we will use will be:

 - **highest predicted return**
 - **lowest risk - the least loss/higher minimum returns among the zipcodes with the highest returns**
 
 
Let's start with that function, and we will use some of the functions that we have created previously as well.

In [ ]:
def rank_return_2year(dictionary_of_zipcodes, city):
    
    '''Takes a dictionary with time series for each zipcode key, and a city name. Calculates returns on investment
    for a 2-year period lag. Model returns and make predictions for future returns. Selects zipcodes with highest
    returns and smaller losses. Plots returns and returns dataframe displaying results.'''
    
    # calculate 2 year returns
    returns = {}
    for i, key in enumerate(dictionary_of_zipcodes):
        temp_return = (dictionary_of_zipcodes[key] - dictionary_of_zipcodes[key].shift(periods=24))/dictionary_of_zipcodes[key].shift(periods=24)*100
        temp_return.dropna(inplace=True)
        returns[list(dictionary_of_zipcodes.keys())[i-1]] = temp_return
        
    # model and get predictions
    results = {}   
    for i, key in enumerate(returns):
        results[list(returns.keys())[i-1]] = {}
        order, seasonal_order = find_orders(returns[key], show=False)
        model = fit_sarimax_model(returns[key], order=order, seasonal_order=seasonal_order, summary=False, plot=False)
        forecast, maximum, minimum = get_predictions(returns[key], model, steps=24, plot=False, show=False)
        results[list(returns.keys())[i-1]][' Current Return'] = round(returns[key].iloc[:-1].values.mean(),0)
        results[list(returns.keys())[i-1]][' Predicted Return'] = forecast
        results[list(returns.keys())[i-1]]['Maximum Return'] = maximum
        results[list(returns.keys())[i-1]]['Minimum Return'] = minimum
    
    # format data into dataframe and filter desired results
    results = pd.DataFrame.from_dict(results)
    
    results = results.transpose()
    
    results = results.nlargest(30, [' Predicted Return'])
    
    results = results.nlargest(5, ['Minimum Return']) 
    
    results = results.nlargest(5, [' Predicted Return'])
    
    # plot results
    ax = results.plot(kind='bar', colormap='Accent', figsize=(8,6))
    plt.axhline(y=0, color='red')
    plt.title(f'Best {city} Zipcodes to Invest for 2-Year Period', fontsize=16)
    ax.spines["top"].set_visible(False)  
    ax.spines["right"].set_visible(False)
    plt.gcf().autofmt_xdate()
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    return results
        

In [ ]:
sf_zips_2yroi = rank_return_2year(sf_zip_dict, city='San Francisco')

In [ ]:
sf_zips_2yroi

In [ ]:
nyc_zips_2yroi = rank_return_2year(nyc_zip_dict, city='New York City')

In [ ]:
nyc_zips_2yroi

In [ ]:
def rank_return_5year(dictionary_of_zipcodes, city):
    
    '''Takes a dictionary with time series for each zipcode key, and a city name. Calculates returns on investment
    for a 5-year period lag. Model returns and make predictions for future returns. Selects zipcodes with highest
    returns and smaller losses. Plots returns and returns dataframe displaying results.'''
    
    returns = {}
    
    # calculate 5 year returns
    for i, key in enumerate(dictionary_of_zipcodes):
        temp_ts = (dictionary_of_zipcodes[key] - dictionary_of_zipcodes[key].shift(periods=60))/dictionary_of_zipcodes[key].shift(periods=60)*100
        temp_ts.dropna(inplace=True)
        returns[list(dictionary_of_zipcodes.keys())[i-1]] = temp_ts
    
    #eliminate data that is too small for forecast
    for key in returns:
        if(len(returns[key]))<=60:
            del returns[key]

    for key in returns:
        if(len(returns[key]))<=60:
            del returns[key]

        
    # model and get predictions
    results = {}   
    for i, key in enumerate(returns):
        results[list(returns.keys())[i-1]] = {}
        order, seasonal_order = find_orders(returns[key], show=False)
        model = fit_sarimax_model(returns[key], order=order, seasonal_order=seasonal_order, summary=False, plot=False)
        forecast, maximum, minimum = get_predictions(returns[key], model, steps=60, plot=False, show=False)
        results[list(returns.keys())[i-1]][' Current Return'] = round(returns[key].iloc[:-1].values.mean(),0)
        results[list(returns.keys())[i-1]][' Predicted Return'] = forecast
        results[list(returns.keys())[i-1]]['Maximum Return'] = maximum
        results[list(returns.keys())[i-1]]['Minimum Return'] = minimum
       
    # format data into dataframe and filter desired results
    results = pd.DataFrame.from_dict(results)
    
    results = results.transpose()
    
    results = results.nlargest(30, [' Predicted Return'])
    
    results = results.nlargest(5, ['Minimum Return']) 
    
    results = results.nlargest(5, [' Predicted Return'])
    
    # plot results
    ax = results.plot(kind='bar', colormap='Accent', figsize=(8,6))
    plt.axhline(y=0, color='red')
    plt.title(f'Best {city} Zipcodes to Invest for 5-Year Period', fontsize=16)
    ax.spines["top"].set_visible(False)  
    ax.spines["right"].set_visible(False)
    plt.gcf().autofmt_xdate()
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    return results

In [ ]:
sf_zips_5yroi = rank_return_5year(sf_zip_dict, city='San Francisco')

In [ ]:
sf_zips_5yroi

In [ ]:
nyc_zips_5yroi = rank_return_5year(nyc_zip_dict, city='New York City')

In [ ]:
nyc_zips_5yroi

## Conclusions and Recommendations

Our stakeholders were interested in investing in real estate either in San Francisco, CA or in New York City, NY. They were looking to find our what was the location that would offer better ROI over a 2-year and a 5-year time invested.

We conducted a time series analysis and modeling based on data from Zillow Research Page, which provides us with the mean Zillow Home Value Index for several years and many zipcodes from both cities. 

Based on our analysis of historical data and on predictions obtained from our models, we would recommend the following:

### **2-Year Investments:**<br>

#### Agressive Investor Profile - Focus on maximum gains: **San Francisco**<br>
   
   
#### Conservative Investor Profile - Focus on minimizing losses: **New York**<br>
   
   5 Best San Francisco Zipcodes for 2-year Investment:  <br>
       - 
       - 
       - 
       - 
       - 
   
   5 Best New York City Zipcodes for 2-year Investment:  <br>
       - 
       - 
       - 
       - 
       - 
   <br>
   
### **5-Year Investments:** city name <br>

   
   5 Best San Francisco Zipcodes for 2-year Investment:  <br>
       - 
       - 
       - 
       - 
       - 
   
   5 Best New York City Zipcodes for 2-year Investment:  <br>
       - 
       - 
       - 
       - 
       - 


## Future Work

For future work we could explore other cities, as well as continue to improve our model by trying to deepen our understanding of the data and trends and continue to tweak and improve our prediction model. We could also experiment with other types of time series models such as Vector Autoregression Moving-Average with Exogenous Regressors (VARMAX), or Holt Winter’s Exponential Smoothing (HWES) for example.

We could also gather more recent data so that our predictions better reflect the current state of real estate in the areas of interest, since the data we worked with goes only up to April 2018.

